# ptime notebook 

Run on Casper full node (36 cores)

In [1]:
%load_ext ptime

In [2]:
import xarray as xr
xr.__version__

'0.18.2'

In [3]:
#from coiled import performance_report
from dask.distributed import performance_report

In [4]:
import dask.array as dsa
import dask
dask.__version__

'2021.06.2'

In [5]:
import numpy as np

np.__version__

'1.18.2'

In [6]:
import gcm_filters as gf
gf.__version__

'0.1'

In [7]:
shape = (160, 1024, 1024)
chunks = (10, 1024, 1024)


nt, ny, nx = shape
da = xr.DataArray(dsa.random.random(shape, chunks=chunks), dims=['time', 'y', 'x'])
mask_data = dsa.ones((ny, nx))
mask_data[(ny // 4):(3 * ny // 4), (nx // 4):(3 * nx // 4)] = 0
wet_mask = xr.DataArray(mask_data, dims=['y', 'x'])

da_masked = da.where(wet_mask)

filter = gf.Filter(
    filter_scale=4,
    dx_min=1,
    filter_shape=gf.FilterShape.TAPER,
    grid_type=gf.GridType.REGULAR_WITH_LAND,
    grid_vars={'wet_mask': wet_mask}
)

da_filtered = filter.apply(da_masked, dims=['y', 'x'])
filter

filter_func = gf.filter._create_filter_func(filter.filter_spec, filter.Laplacian)

data = np.random.rand(10, 1024, 1024)
data_mask = np.random.rand(1, 1024, 1024)

%time filter_func(data, data_mask)

CPU times: user 4.02 s, sys: 2.43 s, total: 6.45 s
Wall time: 6.45 s


array([[[2.61996282, 2.06146906, 2.40964292, ..., 1.60272225,
         2.67882899, 2.80266536],
        [1.32348429, 2.36529716, 1.73208699, ..., 1.9751919 ,
         2.00222161, 2.07322426],
        [0.8567913 , 0.84700807, 1.40015579, ..., 2.34154379,
         2.23924865, 1.95348018],
        ...,
        [1.38697224, 1.34732929, 1.45248767, ..., 0.43835488,
         2.25182984, 1.41219787],
        [1.6227557 , 1.55659304, 1.3098509 , ..., 0.83885408,
         2.11022131, 2.46540641],
        [1.86185938, 1.08980313, 1.55779416, ..., 1.29549619,
         2.01387059, 2.29407065]],

       [[1.65170925, 1.60320078, 2.3416586 , ..., 0.86808774,
         2.34922244, 1.60825505],
        [1.69808049, 1.60367647, 1.63129985, ..., 1.57805553,
         1.80355814, 1.86515566],
        [0.60390745, 1.06168645, 1.48326961, ..., 1.50293806,
         1.6142209 , 1.70651294],
        ...,
        [0.62231405, 1.46568528, 1.57686707, ..., 0.31271431,
         1.4282604 , 1.08808882],
        [1.3

In [8]:
%ptime filter_func(data, data_mask)

Total serial time:   12.88 s
Total parallel time: 6.63 s
For a 1.94X speedup across 2 threads
